In [ ]:
import openpyxl
import csv
import os
from concurrent.futures import ProcessPoolExecutor
from glob import glob

def extract_data_from_sheet(sheet):
    data_name = None
    data_description = None
    data_price = None
    schema_cols = []

    data_product_cells = []
    data_basic_price_cell = None
    table_name_cell = None

    for row in sheet.iter_rows(values_only=False):
        for cell in row:
            if cell.value == "데이터 상품 정보":
                data_product_cells.append(cell)
            elif cell.value == "데이터 기본 이용료":
                data_basic_price_cell = cell
            elif cell.value == "테이블명(한글)":
                table_name_cell = cell

    if len(data_product_cells) == 2:
        right_1 = sheet.cell(row=data_product_cells[0].row, column=data_product_cells[0].column + 1).value
        right_2 = sheet.cell(row=data_product_cells[1].row, column=data_product_cells[1].column + 1).value
        data_name = str(right_1).strip() if right_1 else None
        data_description = str(right_2).strip().replace("\n", "\\n") if right_2 else None

    if data_basic_price_cell:
        price_cell = sheet.cell(row=data_basic_price_cell.row, column=data_basic_price_cell.column + 1)
        data_price = str(price_cell.value).strip() if price_cell.value else None

    if table_name_cell:
        row_idx = table_name_cell.row + 1
        col_idx = table_name_cell.column
        while True:
            cell = sheet.cell(row=row_idx, column=col_idx)
            if cell.value is None or str(cell.value).strip() == "":
                break
            schema_cols.append(str(cell.value).strip())
            row_idx += 1

    return data_name, data_description, data_price, schema_cols


def process_single_file(excel_path, output_dir):
    wb = openpyxl.load_workbook(excel_path, data_only=True)
    filename = os.path.basename(excel_path)
    output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_extracted.csv")

    with open(output_path, "w", newline="", encoding="utf-8-sig") as f_out:
        writer = csv.writer(f_out)
        writer.writerow(["filename", "sheet_name", "data_name", "data_description", "data_price", "schema", "valid"])

        for sheet_name in wb.sheetnames:
            sheet = wb[sheet_name]
            name, desc, price, schema = extract_data_from_sheet(sheet)
            schema_str = "|".join(schema).replace("\n", "\\n")

            is_valid = all([
                name and name.strip(),
                desc and desc.strip(),
                price and price.strip(),
                len(schema) > 0
            ])
            valid_str = "True" if is_valid else "False"

            writer.writerow([filename, sheet_name, name, desc, price, schema_str, valid_str])

    return output_path


def process_all_files_parallel(input_dir, output_dir, max_workers=4):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    excel_files = glob(os.path.join(input_dir, "*.xlsx"))

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for excel_path in excel_files:
            futures.append(executor.submit(process_single_file, excel_path, output_dir))

        for future in futures:
            output_file = future.result()
            print(f"Finished processing: {output_file}")


input_directory = "/home/ubuntu/data_value/bigdata-transportation"
output_directory = "parallel_output_csvs"
process_all_files_parallel(input_directory, output_directory, max_workers=14)


In [ ]:
import pandas as pd
import glob
import os

def merge_csv_files(input_dir, output_file):
    csv_files = glob.glob(os.path.join(input_dir, "*.csv"))
    df_list = []

    for file in csv_files:
        df = pd.read_csv(file, encoding="utf-8-sig")
        df_list.append(df)

    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")

merge_csv_files("parallel_output_csvs", "final_merged_output.csv")


In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os
from concurrent.futures import ProcessPoolExecutor, as_completed

def parse_html_file_to_csv(html_path, output_dir):
    try:
        with open(html_path, "r", encoding="utf-8") as f:
            soup = BeautifulSoup(f, "html.parser")

        name = soup.find("h2", class_="detail-title").get_text(strip=True).replace("︎", "")

        description_tag = soup.find("th", string="상품 요약")
        description = description_tag.find_next("td").get_text(strip=True) if description_tag else ""

        price_tags = soup.find_all("p", class_="price")
        if len(price_tags) > 1:
            price_text = price_tags[1].get_text(strip=True)
        else:
            price_text = price_tags[0].get_text(strip=True) if price_tags else None

        schema_tag = soup.find("th", string="상품 상세설명")
        schema_text = schema_tag.find_next("td").get_text(separator="\n", strip=True) if schema_tag else ""
        schema_lines = [line for line in schema_text.split("\n") if "*" in line]
        schema = [line.replace("\n", "\\n").replace("*", "").strip() for line in schema_lines]

        data = {
            "filename": os.path.basename(html_path),
            "sheet_name": "", 
            "name": name,
            "description": description.replace("\n", "\\n"),
            "price": price_text,
            "schema": "; ".join(schema).replace("\n", "\\n"),
            "valid": True
        }

    except Exception as e:
        print(f"Error processing {html_path}: {e}")
        data = {
            "filename": os.path.basename(html_path),
            "sheet_name": "",
            "name": "",
            "description": "",
            "price": "",
            "schema": "",
            "valid": False
        }

    base_filename = os.path.splitext(os.path.basename(html_path))[0]
    output_path = os.path.join(output_dir, f"{base_filename}_parsed.csv")

    df = pd.DataFrame([data])
    df.to_csv(output_path, index=False, encoding="utf-8-sig")

    return output_path


def find_html_files(root_dir):
    html_files = []
    for dirpath, _, filenames in os.walk(root_dir):
        for fname in filenames:
            if fname.lower().endswith(".html"):
                html_files.append(os.path.join(dirpath, fname))
    return html_files

def parse_html_files_parallel_to_separate_csv(root_dir, output_dir, max_workers=4):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    html_files = find_html_files(root_dir)
    print(f"Found {len(html_files)} HTML files.")

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(parse_html_file_to_csv, path, output_dir): path for path in html_files}

        for future in as_completed(futures):
            output_file = future.result()
            if output_file:
                print(f"Saved: {output_file}")
            else:
                print(f"Failed: {futures[future]}")


root_directory = "/home/ubuntu/data_value/KDX"
output_directory = "separate_parsed_csvs"
parse_html_files_parallel_to_separate_csv(root_directory, output_directory, max_workers=14)


In [ ]:
import pandas as pd
import glob
import os

def merge_csv_files(input_dir, output_file):
    csv_files = glob.glob(os.path.join(input_dir, "*.csv"))
    df_list = []

    for file in csv_files:
        df = pd.read_csv(file, encoding="utf-8-sig")
        df_list.append(df)

    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")

merge_csv_files("separate_parsed_csvs", "final_merged_output_html.csv")


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import traceback


df = pd.read_csv('KDX+국가교통데이터오픈마켓+해양수산.csv', dtype=str)

with open('price-ids', 'r') as f:
    lines = f.readlines()

for line in lines:
    i = line.rstrip()
    filename = f'definition_{i}.xlsx'

    try:
        
        url = f"https://www.bigdata-transportation.kr/frn/prdt/detail?prdtId=PRDTNUM_0000000{i}"

        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        options.add_argument('--no-sandbox')

        driver = webdriver.Chrome(options=options)
        driver.get(url)    

        time.sleep(3) 

        price = driver.find_element(By.CLASS_NAME, 'prd-price').text
        print("Price:", price)

        df.loc[(df['source'] == 'bigdata-transportation') & (df['price'] == '유료') & (df['filename'] == filename) , 'price'] = price

        driver.quit()

        df.to_csv('price-KDX+국가교통데이터오픈마켓+해양수산.csv', index=False)
    except Exception as e:
        print(f'Error on file {filename}')
        traceback.print_exc()


In [ ]:
import pandas as pd

df = pd.read_csv('price-KDX+국가교통데이터오픈마켓+해양수산.csv', dtype=str)

df.loc[(df['source'] == 'bigdata-sea') & (df['price'].isna()), 'price'] = "0"
df.loc[df['price'] == '무료', 'price'] = "0"

df['price'] = df['price'].str.replace(',','')
df['price'] = df['price'].str.replace(' ','')
df['price'] = df['price'].str.replace('원','')

df['collect_dt'] = '2025-07-15'

df = df[df['valid'] == 'True']

df.loc[df['sheet_name'].isna(), 'sheet_name'] = '-'
df.loc[df['schema'].isna(), 'schema'] = ''

df.to_csv('KDX+국가교통데이터오픈마켓+해양수산_20250725.csv', index=False)


In [ ]:
df = pd.read_csv('KDX+국가교통데이터오픈마켓+해양수산_20250725.csv', dtype=str)

df.loc[df['schema'].isna(), 'schema'] = ''

df['sorted_schema'] = (
    df['schema'].str.split('|')
    .apply(sorted)
    .apply(lambda x: '-'.join(x))
)

df.to_csv('KDX+국가교통데이터오픈마켓+해양수산_20250725_sorted.csv', index=False)

In [ ]:
import pandas as pd

df = pd.read_csv('/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250725_sorted.csv')
df['company'] = '-'
df['category'] = '-'
df['extension'] = '-'

df

In [ ]:
import os

from bs4 import BeautifulSoup



html_dir = '/home/ubuntu/data_value/KDX'
results = []

for filename in os.listdir(html_dir):

     
    if not filename.endswith('.html'):
        continue

    file_path = os.path.join(html_dir, filename)

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            html = f.read()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='euc-kr') as f:
            html = f.read()

    soup = BeautifulSoup(html, 'html.parser')

    # 초기화
    category = None
    extension = None
    company = None

    for row in soup.select('table.table-detail tr'):
        th = row.find('th')
        td = row.find('td')
        if not th or not td:
            continue

        label = th.get_text(strip=True)
        value = td.get_text(strip=True)

        if label == '상품 카테고리':
            category = value
        elif label == '확장자':
            extension = value
        elif label == '회사명':
            company = value

    item = {
        '파일명': filename,
        '카테고리': category,
        '확장자': extension,
        '회사명': company
    }

    df.loc[df['filename'] == filename, 'company'] = company
    df.loc[df['filename'] == filename, 'category'] = category
    df.loc[df['filename'] == filename, 'extension'] = extension

    results.append(item)


    print(item)


In [ ]:
import pandas as pd

df = pd.read_csv('/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250730_1.csv', dtype=str)

df['tags'] = '-'

In [ ]:
import os
from bs4 import BeautifulSoup

# Directory where HTML files are stored
html_dir = '/home/ubuntu/data_value/KDX'
all_tags = []

# Walk through each HTML file in the directory
for filename in os.listdir(html_dir):
    if filename.endswith('.html'):
        with open(os.path.join(html_dir, filename), 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'html.parser')
            # Find all divs with class 'tag' and attribute 'data-text'
            tag_divs = soup.find_all('div', class_='tag', attrs={'data-text': True})
            tags = [tag['data-text'] for tag in tag_divs]
            all_tags.extend(tags)

            print(tags)            
            df.loc[df['filename'] == filename, 'tags'] = str(tags)

In [ ]:
df[df['category'].isna()]

In [ ]:
df.to_csv('/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250730_2.csv', index=False)

In [ ]:
df['category'].unique()

In [ ]:
{
    "도로": {
        "차량": ["위치", "속도"]
    },
    "도시": {
        "상권": ["학원", "카페", "편의점"],
        "환경": ["기온", "대기"],
        "인구": ["인구세대"],
        "이동": ["유동인구"],
        "재무": ["소득/소비", "부채", "신용", "대출", "자산"]
    }
}

In [ ]:
import pandas as pd
from collections import Counter
import re

df = pd.read_csv("/home/ubuntu/data_value/KDX+국가교통데이터오픈마켓+해양수산_20250730_1.csv")  # make sure 'description' column exists

# Clean and tokenize
def tokenize(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    return text.split()

df['tokens'] = df['description'].apply(lambda x: tokenize(x) if isinstance(x, str) else [])


# Flatten all tokens
all_tokens = [word for tokens in df['tokens'] for word in tokens]
word_freq = Counter(all_tokens)

# Top 50 most common words
top_words = word_freq.most_common(10000)
for word, freq in top_words:
    print(f"{word}: {freq}")